In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from openai import OpenAI
import uvicorn
from datetime import datetime
nest_asyncio.apply()
app = FastAPI()
llm_api_key="sk-a990af2e87ee4a6b8d94195520cf2a70"#填入Api_Key
class ChatRequest(BaseModel):
    message: str
    studentId: str
    chatHistoryId: int

In [2]:
import faiss
import json
import numpy as np
from text2vec import SentenceModel
import jieba.analyse
model = SentenceModel("BAAI/bge-large-zh-v1.5")
title_index = faiss.read_index("title.index")
desc_index = faiss.read_index("desc.index")
with open("index2title.json", "r", encoding="utf-8") as f:
    titles = json.load(f)

2025-06-22 08:50:13.445 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: cuda


In [3]:
#mysql链接
from dbutils.pooled_db import PooledDB
import pymysql
pool = PooledDB(
    creator=pymysql,
    maxconnections=5,
    blocking=True,
    host='localhost',
    user='root',
    password='root',
    database='chatbot',
    charset='utf8mb4',
    autocommit=True
)
def get_conn():
    return pool.connection()

In [4]:
#格式清理
import json, html
from html import unescape
from bs4 import BeautifulSoup
def strip_html(text:str) -> str:
    return BeautifulSoup(text, "html5lib").get_text(" ", strip=True)
def safe_text(value):
    """None/空串/空容器 统一返回 '无'，同时去 html & 转义"""
    if value in (None, "", [], {}, ()):
        return "无"
    if isinstance(value, (list, tuple)):
        return "、".join(str(v) for v in value)
    text = strip_html(str(value))
    return html.unescape(text)

def format_solution(solution_field):
    if not solution_field:
        return "无"
    if isinstance(solution_field, str):
        try:
            solution_field = json.loads(solution_field)
        except Exception:
            code_plain = html.unescape(solution_field)
            return f"```\n{code_plain}\n```"
    if isinstance(solution_field, dict):
        solution_field = [solution_field]
    if not isinstance(solution_field, list):
        return f"```text\n{str(solution_field)}\n```"
    blocks = []
    for item in solution_field:
        lang = item.get("language", "代码")
        code = html.unescape(item.get("code", ""))
        blocks.append(f"语言：{lang}\n```{lang.lower()}\n{code}\n```")
    return "\n\n".join(blocks)
def build_knowledge(record: dict) -> str:
    title             = safe_text(record.get("title"))
    description       = safe_text(record.get("description"))
    in_desc           = safe_text(record.get("input_description"))
    out_desc          = safe_text(record.get("output_description"))
    sample_in         = safe_text(record.get("sample_input"))
    sample_out        = safe_text(record.get("sample_output"))
    try:
        sample_in  = safe_text(json.loads(sample_in))
    except Exception:
        pass
    try:
        sample_out = safe_text(json.loads(sample_out))
    except Exception:
        pass

    solution=format_solution(record.get("solution"))
    knowledge = (
        f"【题目标题】\n{title}\n\n"
        f"【题目描述】\n{description}\n\n"
        f"【输入说明】\n{in_desc}\n\n"
        f"【输出说明】\n{out_desc}\n\n"
        f"【样例输入】\n{sample_in}\n\n"
        f"【样例输出】\n{sample_out}\n\n"
        f"【题目解析】\n{solution}"
    )
    return knowledge

In [5]:
#MySQL模糊查询
import re
# 清理语气/提示词
QUESTION_PATTERNS = [
    r"这个题目怎么(做|实现)", r"该怎么(实现|做)", r"怎么实现", r"怎么做",
    r"请帮我实现", r"题目是", r"这道题", r"如何实现", r"应该怎么写"
]

def clean_user_input(user_input: str) -> str:
    if not isinstance(user_input, str):
        user_input = str(user_input)
    for pattern in QUESTION_PATTERNS:
        user_input = re.sub(pattern, '', user_input, flags=re.IGNORECASE)
    return user_input.strip()

# 提取部分 n-gram 子串
def extract_ngrams(text: str, min_len=6, max_len=12):
    ngrams = set()
    length = len(text)
    for l in range(min_len, max_len + 1):
        for i in range(length - l + 1):
            ngrams.add(text[i:i+l])
    return list(ngrams)

# 查询函数：只返回一个字符串或 None
def find_first_title(user_input: str) -> str | None:
    cleaned = clean_user_input(user_input)
    conn = get_conn()
    cursor = conn.cursor()

    try:
        # 全文匹配
        match_sql = """
        SELECT title FROM problem
        WHERE MATCH(title, description) AGAINST (%s IN NATURAL LANGUAGE MODE)
        LIMIT 1
        """
        cursor.execute(match_sql, (cleaned,))
        result = cursor.fetchone()
        if result:
            return str(result[0])

        # LIKE 匹配
        like_sql = """
        SELECT title FROM problem
        WHERE title LIKE %s OR description LIKE %s
        LIMIT 1
        """
        like_pattern = f"%{cleaned}%"
        cursor.execute(like_sql, (like_pattern, like_pattern))
        result = cursor.fetchone()
        if result:
            return str(result[0])

        # n-gram 模糊匹配
        ngrams = extract_ngrams(cleaned)
        for gram in ngrams:
            pattern = f"%{gram}%"
            cursor.execute(like_sql, (pattern, pattern))
            result = cursor.fetchone()
            if result:
                return str(result[0])

        return None
    finally:
        cursor.close()
        conn.close()


In [6]:
#向量检索
def preprocess_and_vectorize(query, model):
    keywords = jieba.analyse.extract_tags(query, topK=5)
    query_core = " ".join(keywords) if keywords else query
    return model.encode([query_core], normalize_embeddings=True)
def search_top_title(query: str) -> str | None:
    query_vec = preprocess_and_vectorize(query, model)
    D1, I1 = title_index.search(np.array(query_vec), k=5)
    D2, I2 = desc_index.search(np.array(query_vec), k=5)
    # 加权得分合并
    score_map = {}
    for idx, score in zip(I1[0], D1[0]):
        title = titles[idx]
        score_map[title] = score_map.get(title, 0) + 0.6 * score
    for idx, score in zip(I2[0], D2[0]):
        title = titles[idx]
        score_map[title] = score_map.get(title, 0) + 0.4 * score
    # 过滤得分过低的匹配
    MIN_SCORE = 0.7
    top_k = [(title, score) for title, score in score_map.items() if score >= MIN_SCORE]
    top_k = sorted(top_k, key=lambda x: x[1], reverse=True)
    # 返回得分最高的标题，或 None
    return top_k[0][0] if top_k else None

In [8]:
#获取查询到的标题
import ast
def get_top_title(user_input,chat_history_id):
    try:
        parsed = ast.literal_eval(user_input)
        if isinstance(parsed, list) and len(parsed) > 0:
            user_input = parsed[0].get("content", "")
    except Exception as e:
        print(f"[警告] user_input 内容格式不是标准结构：{e}")
    #优先sql查询
    keyword = find_first_title(user_input)
    #若sql查询失败，再向量查询
    if keyword is None:
        #print("sql查询失败")
        keyword = search_top_title(user_input) 
    #如果都失败，表明查询失败
    if keyword is None:
        try:
            conn = get_conn()
            cursor = conn.cursor()
            cursor.execute("SELECT knowledge FROM chat_history WHERE id = %s", (chat_history_id,))
            result = cursor.fetchone()
            if result and result[0]:
                keyword = result[0]
            cursor.close()
            conn.close()
        except Exception as e:
            print(f"[错误] 查询 chat_history 失败：{e}")
            keyword = None
        return keyword
    save_keyword_to_history(chat_history_id, keyword)
    return keyword

In [9]:
#将keyword保存到chat_history表中的knowledge字段
def save_keyword_to_history(chat_history_id, keyword):
    if keyword is None:
        return  
    conn = get_conn()
    cursor = conn.cursor()
    update_sql = "UPDATE chat_history SET knowledge = %s WHERE id = %s"
    cursor.execute(update_sql, (keyword, chat_history_id))
    conn.commit()
    cursor.close()
    conn.close()

In [10]:
#从数据库中提取对应记录并构造knowledge文本段
def generate_knowledge_from_db(keyword):
    if keyword is None:
        return "无相关知识"
    conn = get_conn()
    cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
    query_sql = """
        SELECT title, description, input_description, output_description, 
               sample_input, sample_output, solution 
        FROM problem WHERE title = %s
    """
    cursor.execute(query_sql, (keyword,))
    result = cursor.fetchone()
    cursor.close()
    conn.close()
    if not result:
        return "无相关知识"
    for key in result:
        val = result[key]
        if val in (None, "", [], {}):
            result[key] = "无"
    return build_knowledge(result)

In [11]:
#RAG功能模块
def RAG_model(user_input,chat_history_id):
    keyword = get_top_title(user_input,chat_history_id)
    print(keyword)
    knowledge = generate_knowledge_from_db(keyword)
    return knowledge

In [12]:
#用户画像绘制
from datetime import datetime, timedelta, time
from zoneinfo import ZoneInfo
from typing import List, Dict
from contextlib import closing
def user_added_message_summary(user_id):
    tz_bj = ZoneInfo("Asia/Shanghai")
    today_bj = datetime.now(tz_bj).date()         
    yesterday = today_bj - timedelta(days=1)        

    start_dt = datetime.combine(yesterday, time.min, tzinfo=tz_bj)   
    end_dt   = datetime.combine(today_bj, time.min,   tzinfo=tz_bj)  

    sql = """
        SELECT cd.user_chat, cd.ai_chat
        FROM   chat_details  AS cd
        JOIN   chat_history  AS ch ON cd.chat_history_id = ch.id
        WHERE  ch.user_id = %s
          AND  cd.create_time >= %s
          AND  cd.create_time <  %s
        ORDER BY cd.create_time;
    """

    rows: List[tuple] = []
    with closing(get_conn()) as conn, closing(conn.cursor()) as cur:
        cur.execute(sql, (user_id, start_dt, end_dt))
        rows = cur.fetchall()
    messages: List[Dict[str, str]] = []
    for user_chat, ai_chat in rows:
        if user_chat:
            messages.append({"role": "user", "content": user_chat})
        if ai_chat:
            messages.append({"role": "assistant", "content": ai_chat})

    return messages

In [13]:
def user_draw():
    conn = get_conn()
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT id, user_draw FROM users")
        users = cursor.fetchall()

        for user_id, old_draw in users:
            message = user_added_message_summary(user_id)
            prompt = {"role": "user", "content": "用户是一位学生，请基于上述信息生成该用户的画像。包含该用户对于AI语气及教学方式的偏好，用户可能的性格倾向，以及其他可能的可以提高AI应答质量的信息"}
            message.append(prompt)
            if old_draw:
                message.insert(0, {"role": "system", "content": f"以下是用户的旧画像，请一并参考：{old_draw}"})
            client = OpenAI(api_key=llm_api_key, base_url="https://api.deepseek.com")
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=message,
                temperature=0.7         
            )
            new_draw = response.choices[0].message.content.strip()
            update_sql = "UPDATE users SET user_draw = %s WHERE id = %s"
            cursor.execute(update_sql, (new_draw, user_id))
        conn.commit()
        print("所有用户画像已成功更新。")
    except Exception as e:
        conn.rollback()
        print(f"出现错误: {e}")
    finally:
        cursor.close()
        conn.close()

In [14]:
#上下文归纳
def context_summary(chat_history_id):
    context=context_chained(chat_history_id)
    prompt = f"请对以下对话内容进行简洁总结，在尽可能保存关键信息的前提下尽量的压缩文本长度，用中文表述重点信息：\n\n{context}"
    try:
        client = OpenAI(api_key=llm_api_key, base_url="https://api.deepseek.com")
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "你是一个善于总结信息的助手。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
            
        )
        summary = response.choices[0].message.content.strip()
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        conn = get_conn()
        with conn.cursor() as cursor:
            insert_sql = """
                INSERT INTO chat_summary (chat_history_id, summary, create_time)
                VALUES (%s, %s, %s)
            """
            cursor.execute(insert_sql, (chat_history_id, summary, now))
            update_sql = """
                UPDATE chat_history
                SET summary_count = summary_count + 1
                WHERE id = %s
            """
            cursor.execute(update_sql, (chat_history_id,))
            update_context_sql = """
                UPDATE chat_history
                SET context_count = GREATEST(context_count - 10, 0)
                WHERE id = %s
            """
            cursor.execute(update_context_sql, (chat_history_id,))

        conn.close()
        print("总结保存成功，summary_count 已加 1")

    except Exception as e:
        print("总结失败:", e)

In [15]:
#二次归纳
def secondary_summary(chat_history_id):   
    conn = get_conn()
    try:
        with conn.cursor() as cursor:

            select_sql = """
                SELECT summary FROM chat_summary
                WHERE chat_history_id = %s
                ORDER BY create_time ASC
            """
            cursor.execute(select_sql, (chat_history_id,))
            rows = cursor.fetchall()
            context = "\n\n".join([row[0] for row in rows])
            prompt = f"以下是多个总结，请进行高阶总结，在尽可能保存关键信息的前提下尽量的压缩文本长度，提炼最核心的内容：\n\n{context}"
            client = OpenAI(api_key=llm_api_key, base_url="https://api.deepseek.com")
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一个善于总结信息的助手。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7
            
            )
            final_summary = response.choices[0].message.content.strip()
            now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            delete_sql = "DELETE FROM chat_summary WHERE chat_history_id = %s"
            cursor.execute(delete_sql, (chat_history_id,))
            insert_sql = """
                INSERT INTO chat_summary (chat_history_id, summary, create_time)
                VALUES (%s, %s, %s)
            """
            cursor.execute(insert_sql, (chat_history_id, final_summary, now))
            update_sql = """
                UPDATE chat_history
                SET summary_count = 1
                WHERE id = %s
            """
            cursor.execute(update_sql, (chat_history_id,))

        conn.close()
        print("二次总结完成并保存成功")

    except Exception as e:
        print("二次总结失败:", e)

In [16]:
#上下文拼接
def context_chained(chat_history_id):
    conn = get_conn()
    context = []
    try:
        with conn.cursor() as cursor:
            sql = """
                SELECT user_chat, ai_chat 
                FROM chat_details 
                WHERE chat_history_id = %s 
                ORDER BY create_time DESC 
                LIMIT 10
            """
            cursor.execute(sql, (chat_history_id,))
            rows = cursor.fetchall()

            # 倒序转为正序，形成自然对话顺序
            for row in reversed(rows):
                user_msg, ai_msg = row
                context.append({"role": "user", "content": user_msg})
                context.append({"role": "assistant", "content": ai_msg})
                
    except Exception as e:
        print("上下文拼接失败:", e)
    finally:
        conn.close()

    return context

In [17]:
def summary_chained(chat_history_id):
    conn = get_conn()
    try:
        with conn.cursor() as cursor:
            select_sql = """
                SELECT summary FROM chat_summary
                WHERE chat_history_id = %s
                ORDER BY create_time ASC
            """
            cursor.execute(select_sql, (chat_history_id,))
            rows = cursor.fetchall()
            # 提取文本内容并拼接
            summaries = [row[0] for row in rows if row[0]]
            combined = "\n\n".join(summaries)
            return combined if combined else None

    except Exception as e:
        print("summary_chained 查询失败:", e)
        return None
    finally:
        conn.close()

In [18]:
def get_user_draw(user_id):
    conn = get_conn()
    cursor = conn.cursor()
    try:
        # 查询该用户的画像
        sql = "SELECT user_draw FROM users WHERE id = %s"
        cursor.execute(sql, (user_id,))
        result = cursor.fetchone()

        # 构造 LLM API 所需格式
        if result and result[0]:
            content = result[0]
        else:
            content = "不存在用户画像"

        messages = [
            {"role": "system", "content": content}
        ]
        return messages

    except Exception as e:
        print(f"查询用户画像失败: {e}")
        return [{"role": "system", "content": "查询用户画像出错"}]
    finally:
        cursor.close()
        conn.close()

In [19]:
#messages汇总拼接
def messages_chained(user_input, chat_history_id,user_id):
    # 提示词
    promat = """你是一名优秀且耐心的C/C++语言编程教师，用户是一位正在学习编程的初学者。请严格遵循以下教学原则：

1. 你的主要任务是针对用户提出的问题，通过引导式的方法去回答用户的疑问
2. 如果用户没有提出具体的问题，你可以列举一些示例，鼓励用户的提问；
3. 你绝不直接提供任何真实的代码
4. 你可以在合适时机提供关键逻辑的伪代码，帮助学生建立思路，但要避免替用户实现细节；
5. 你应使用简单直观的例子（如加减法、循环遍历）来解释复杂概念；
6. 若用户提出与编程无关的问题，请礼貌拒绝，并引导回到学习主题；
7. 你要用简单易懂，符合入门大学生理解能力的方式去引导用户
8. 保持清晰的输出格式，以便用户的阅读

请始终保持语气鼓励、清晰，充满活力，帮助学生逐步掌握解决问题的方法，而不是替他们写代码。
"""
    # 获取上下文（最近10轮）
    context = context_chained(chat_history_id)
    # 获取总结信息
    summary = summary_chained(chat_history_id)
    # 获取knowledge
    knowledge=RAG_model(user_input,chat_history_id)
    # 获取用户画像
    user_draw=get_user_draw(user_id)
    # 构造完整 messages 列表
    messages = [{"role": "system", "content": promat}]
    messages.extend(user_draw)
    if summary:
        messages.append({"role": "system", "content": f"背景信息：{summary}"})
    if knowledge:
        messages.append({"role": "system", "content": f"相关知识，如果你认为不必要可以忽略{knowledge}"})
    messages.extend(context)
    messages.append({"role": "user", "content": user_input})
    return messages

In [20]:
def memory_save(user_input, ai_reply, chat_history_id):
    conn = get_conn()
    try:
        with conn.cursor() as cursor:
            insert_sql = """
                INSERT INTO chat_details (chat_history_id, user_chat, ai_chat, create_time)
                VALUES (%s, %s, %s, %s)
            """
            now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cursor.execute(insert_sql, (chat_history_id, user_input, ai_reply, now))

            # 更新 context_count +1
            update_sql = """
                UPDATE chat_history
                SET context_count = context_count + 1
                WHERE id = %s
            """
            cursor.execute(update_sql, (chat_history_id,))

            print("保存成功，context_count 已加 1")
            conn.commit()
        with conn.cursor() as cursor:
            cursor.execute(
                "SELECT context_count FROM chat_history WHERE id = %s",
                (chat_history_id,)
            )
            context_count = cursor.fetchone()[0]
        if context_count >= 10:
            print("触发 context_summary")
            context_summary(chat_history_id)
            with conn.cursor() as cursor:
                cursor.execute(
                    "SELECT summary_count FROM chat_history WHERE id = %s",
                    (chat_history_id,)
                )
                summary_count = cursor.fetchone()[0]

            if summary_count >= 10:
                print("触发 secondary_summary")
                secondary_summary(chat_history_id)

    except Exception as e:
        print("保存失败:", e)
    finally:
        conn.close()

In [21]:
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from openai import OpenAI

app = FastAPI()

@app.post("/chat/ask")
async def chat_response(request: Request):
    body = await request.json()

    student_id = body.get("studentId")
    chat_history_id = body.get("chatHistoryId")
    user_input = body.get("messages")
    user_input=user_input[-1]['content']
    user_input=str(user_input)
    chat_history_id=int(chat_history_id)
    student_id=str(student_id)
    message = messages_chained(user_input, chat_history_id,student_id)
    client = OpenAI(api_key=llm_api_key, base_url="https://api.deepseek.com")
    print(message)
    def generate():
        ai_reply = ""
        try:
            stream = client.chat.completions.create(
                model="deepseek-chat",
                messages=message,
                stream=True
            )
            for chunk in stream:
                content = chunk.choices[0].delta.content
                if content:
                    ai_reply += content
                    yield content
        finally:
            memory_save(user_input, ai_reply,chat_history_id)

    return StreamingResponse(generate(), media_type="text/plain")

In [25]:
#触发后会更新用户画像，可以暂不触发
user_draw()

所有用户画像已成功更新。


In [22]:
# 启动服务（非阻塞）
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [5740]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


[警告] user_input 内容格式不是标准结构：invalid character '？' (U+FF1F) (<unknown>, line 1)
欧拉函数
[{'role': 'system', 'content': '你是一名优秀且耐心的C/C++语言编程教师，用户是一位正在学习编程的初学者。请严格遵循以下教学原则：\n\n1. 你的主要任务是针对用户提出的问题，通过引导式的方法去回答用户的疑问\n2. 如果用户没有提出具体的问题，你可以列举一些示例，鼓励用户的提问；\n3. 你绝不直接提供任何真实的代码\n4. 你可以在合适时机提供关键逻辑的伪代码，帮助学生建立思路，但要避免替用户实现细节；\n5. 你应使用简单直观的例子（如加减法、循环遍历）来解释复杂概念；\n6. 若用户提出与编程无关的问题，请礼貌拒绝，并引导回到学习主题；\n7. 你要用简单易懂，符合入门大学生理解能力的方式去引导用户\n8. 保持清晰的输出格式，以便用户的阅读\n\n请始终保持语气鼓励、清晰，充满活力，帮助学生逐步掌握解决问题的方法，而不是替他们写代码。\n'}, {'role': 'system', 'content': '不存在用户画像'}, {'role': 'system', 'content': '相关知识，如果你认为不必要可以忽略【题目标题】\n欧拉函数\n\n【题目描述】\n在数论中，欧拉函数f(n)被定义为：小于等于n的正整数中和n互质的数的数目（互质即两者最大公约数为1） 如f(1)=1，因为与小于等于1的数中与1互质的数只有1 再如f(8)=4，因为小于等于8中的数与8互质的数有1,3,5,7，而2与8最大公约数为2，4与8最大公约数为4，6与8最大公约数为2 现在给定一个数n，求f(n)的值 要求定义并调用函数f(n)，返回欧拉函数的值\n\n【输入说明】\n测试样例有多组，第一行输入一个整数T，代表测试组数 接下来的每一行输入一个整数n，代表需要计算f(n)的n\n\n【输出说明】\n对于每组测试样例，各输出一行，其中包括一个整数f(n)\n\n【样例输入】\n3\n1\n4\n8\n\n\n【样例输出】\n1\n2\n4\n\n\n【题目解析】\n```\n[{\'language\': \'C\', \'code\': \'#include <std

C:\Users\86180\AppData\Local\Temp\ipykernel_5740\3359884574.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html5lib").get_text(" ", strip=True)


保存成功，context_count 已加 1
[警告] user_input 内容格式不是标准结构：invalid character '，' (U+FF0C) (<unknown>, line 1)
欧拉函数
[{'role': 'system', 'content': '你是一名优秀且耐心的C/C++语言编程教师，用户是一位正在学习编程的初学者。请严格遵循以下教学原则：\n\n1. 你的主要任务是针对用户提出的问题，通过引导式的方法去回答用户的疑问\n2. 如果用户没有提出具体的问题，你可以列举一些示例，鼓励用户的提问；\n3. 你绝不直接提供任何真实的代码\n4. 你可以在合适时机提供关键逻辑的伪代码，帮助学生建立思路，但要避免替用户实现细节；\n5. 你应使用简单直观的例子（如加减法、循环遍历）来解释复杂概念；\n6. 若用户提出与编程无关的问题，请礼貌拒绝，并引导回到学习主题；\n7. 你要用简单易懂，符合入门大学生理解能力的方式去引导用户\n8. 保持清晰的输出格式，以便用户的阅读\n\n请始终保持语气鼓励、清晰，充满活力，帮助学生逐步掌握解决问题的方法，而不是替他们写代码。\n'}, {'role': 'system', 'content': '不存在用户画像'}, {'role': 'system', 'content': '相关知识，如果你认为不必要可以忽略【题目标题】\n欧拉函数\n\n【题目描述】\n在数论中，欧拉函数f(n)被定义为：小于等于n的正整数中和n互质的数的数目（互质即两者最大公约数为1） 如f(1)=1，因为与小于等于1的数中与1互质的数只有1 再如f(8)=4，因为小于等于8中的数与8互质的数有1,3,5,7，而2与8最大公约数为2，4与8最大公约数为4，6与8最大公约数为2 现在给定一个数n，求f(n)的值 要求定义并调用函数f(n)，返回欧拉函数的值\n\n【输入说明】\n测试样例有多组，第一行输入一个整数T，代表测试组数 接下来的每一行输入一个整数n，代表需要计算f(n)的n\n\n【输出说明】\n对于每组测试样例，各输出一行，其中包括一个整数f(n)\n\n【样例输入】\n3\n1\n4\n8\n\n\n【样例输出】\n1\n2\n4\n\n\n【题目解析】\n```\n[{\'language\': \'C\', \

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5740]
